# 1. Data collection

## 1.1 Get the list of animes

In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
def getUrls(start, stop): 

    urls = []

    for i in range(start, stop):
        url = 'https://myanimelist.net/topanime.php?limit='+str(i*50)
        r = requests.get(url)
        html_content = r.text
        soup = bs(html_content, 'lxml')
        links = soup.find_all('h3') 

        for anime in links[:-3]:
            if anime.find('a'):
                urls.append(anime.find('a')['href'])
    return urls

In [3]:
### PROVAAAAAA

f = open("topAnime.txt", 'w', encoding="utf8")
f.write('\n'.join(getUrls(0,20)))
f.close

<function TextIOWrapper.close()>

## 1.2 Crawl animes

In [4]:
import os

def saveHtml(page):
#saving the HTML file of page 'page' in the corresponding folder

    subfolder = "downloaded_Html/page_{}".format(page)
    os.makedirs(subfolder)

    f = open("topAnime.txt", 'r', encoding="utf8")
    lines = f.readlines()[(page-1)*50:(page)*50]
    lines = [line.rstrip() for line in lines]
    f.close

    i = 1+50*(page-1)
    for link in lines:
        html = requests.get(link)
        file_name = '{}/{}.html'.format(subfolder, i)
        g = open(file_name, 'w', encoding="utf8")
        g.write(html.text)
        g.close
        i += 1

In [5]:
#### PROVAAAAA

saveHtml(1)

## 1.3 Parse downloaded pages

In [308]:
def getTitle(anime):
    return anime.strong.contents[0]

In [309]:
def getType(anime):
    return anime.find(text = 'Type:').find_next('a').contents[0]

In [310]:
def getNumEpis(anime):
    return int(anime.find(text = 'Episodes:').next_element.strip())

In [311]:
def getStart(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    if len(date)>15:
        return dt.strptime(date.split(" to ")[0], '%b %d, %Y' )
    else:
        return dt.strptime(date, '%b %d, %Y' )

In [312]:
def getEnd(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    if len(date)>15:
        return dt.strptime(date.split(" to ")[1], '%b %d, %Y' )
    else:
        return pd.to_datetime(np.NaN, errors='coerce')

In [313]:
def getNumMemb(anime):
    animeNumMembers = anime.find(text = 'Members:').next_element
    return int(animeNumMembers.replace('n', '').replace(',', '').strip())

In [314]:
def getScore(anime):  
    animeScore = anime.find(text = 'Score:').find_next('span').contents
    return float(animeScore[0])

In [315]:
def getUsers(anime):
    animeUsers = anime.find(text = 'Score:').find_next('span').find_next('span').contents
    return int(animeUsers[0])

In [316]:
def getRank(anime):
    animeRank = anime.find(text = 'Ranked:').next_element
    return int(animeRank.replace('\n', '').replace('#', '').strip())

In [317]:
def getPopularity(anime):
    animePopularity = anime.find(text='Popularity:').next_element
    return int(animePopularity.replace("\n","").replace('#', '').strip())

In [318]:
def getDescription(anime):
    animeDescription = anime.find(text = 'Synopsis').find_next('p').text
    return animeDescription.replace("\n","")

In [319]:
def getRelated(anime):
    table = anime.find(text = 'Related Anime').find_next('table')
    table = table.find_all('a')
    animeRelated = []

    for el in table:
        animeRelated.append(el.text)
                
    return animeRelated

In [320]:
def getCharact(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('table')

    animeChar = []

    for el in table:
        people = el.find_all('h3')
        for person in people:
            animeChar.append(person.text)
        
    return animeChar

In [321]:
def getVoices(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('h3')

    animeVoices = []

    for el in table:
        people = el.find_next('table')
        for person in people:
            animeVoices.append(person.find('a').text)
        
    return animeVoices

In [372]:
def getStaff(anime):
    table = anime.find_all("div", {"class": "detail-characters-list clearfix"})
    if len(table)>1:
        table = table[1]
    table = table.find_all("table")

    animeStaff = []
    for el in table:
        x = el.find_all("td")[1]
        person = [x.find("a").text, x.find("small").text]
        animeStaff.append(person)
    
    return animeStaff

In [384]:
from datetime import datetime as dt
import numpy as np

pages = os.listdir('downloaded_Html')[1:]

title = []
typ = []
numEpisode = []
start = []
end = []
numMembers = []
score = []
users = []
rank = []
popularity = []
synopsis = []
related = []
char = []
voices = []
staff  = []

In [385]:
for page in pages:
    htmls = os.listdir('downloaded_Html/{}'.format(page))
    for i in range(1,1+len(htmls)):
        f = open("downloaded_Html/{}/{}.html".format(page,i), 'r', encoding="utf8")
        anime = bs(f, 'lxml')
        
        title.append(getTitle(anime))
        typ.append(getType(anime))
        numEpisode.append(getNumEpis(anime))
        start.append(getStart(anime))
        end.append(getEnd(anime))
        numMembers.append(getNumMemb(anime))
        score.append(getScore(anime))
        users.append(getUsers(anime))
        rank.append(getRank(anime))
        popularity.append(getPopularity(anime))
        synopsis.append(getDescription(anime))
        related.append(getRelated(anime))
        char.append(getCharact(anime))
        voices.append(getVoices(anime))
        staff.append(getStaff(anime))

1
2
3
4


In [386]:
import pandas as pd 

col = ['animeTitle', 'animeType', 'animeNumEpisode','releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 
       'animeRank', 'animePopularity', 'animdeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

types = {'animeTitle' : 'object', 
         'animeType' : 'object', 
         'animeNumEpisode' : 'int64',
         'releaseDate' : 'datetime64', 
         'endDate' : 'datetime64', 
         'animeNumMembers' : 'int64', 
         'animeScore' : 'float64',
         'animeUsers' : 'int64', 
         'animeRank' : 'int64',
         'animePopularity' : 'int64',
         'animdeDescription' : 'object',
         'animeRelated' : 'object',
         'animeCharacters' : 'object',
         'animeVoices' : 'object',
         'animeStaff' : 'object'}

data = list(zip(title,typ,numEpisode,start,end,numMembers,score,users,rank,popularity,synopsis,related,char,voices,staff))

df = pd.DataFrame(data, columns = col).astype(dtype = types)  

[('Fullmetal Alchemist: Brotherhood',
  'TV',
  64,
  datetime.datetime(2009, 4, 5, 0, 0),
  datetime.datetime(2010, 7, 4, 0, 0),
  2677691,
  9.16,
  1629051,
  1,
  3,
  'After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse\'s body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse\'s soul in the physical realm by binding it to a hulking suit of armor.The brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore th

In [368]:
def dataframeToTSV()

SyntaxError: invalid syntax (<ipython-input-368-2fc876983959>, line 1)